# Explicamelo como si tuviera 5 años


![Architecture](images/architecture.png)

En este cuaderno vamos a recorrer el proceso de configurar un chatbot simple en LangChain.

A lo largo de este proceso, vamos a mostrar cómo LangSmith puede usarse para mejorar la experiencia del desarrollador en aplicaciones de IA.

Empecemos cargando nuestras variables de entorno desde el archivo .env.

In [12]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)
# Cargar las siguientes variables de entorno:
# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_PROJECT="eli5-bot"
# LANGSMITH_API_KEY="<redacted>"

# OPENAI_API_KEY="<redacted>"
# TAVILY_API_KEY="<redacted>"

True

## Setup

Configuremos una herramienta llamada Tavily para permitir que nuestro asistente busque en la web al responder.

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

Diseñemos un prompt para RAG que usaremos a lo largo de todo el cuaderno.

In [14]:
prompt = """Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:"""
print("Prompt Template: ", prompt)

Prompt Template:  Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:


## Creando nuestra aplicación

In [15]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    formatted = prompt.format(question=question, context=context)
    
    completion = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": formatted},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )
    return completion.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

## Testeando nuestra application

In [16]:
question = "Qué es la macroeconomía?"
print(eli5(question))

Imagina que la economía de un país es como un enorme parque de diversiones. En este parque, hay muchas atracciones, como montañas rusas, juegos de agua y carruseles. Ahora, la macroeconomía es como un grupo de personas que mira todo el parque desde arriba, en lugar de enfocarse solo en una atracción específica.

La macroeconomía estudia cosas grandes y importantes sobre el parque entero, como cuántas personas vienen a divertirse, cuántas atracciones hay, si hay suficiente comida y bebida para todos, y si las entradas son baratas o caras. También se fija en si las personas están felices y trabajadoras para ayudar a que el parque funcione bien.

Así que, en resumen, la macroeconomía es el estudio de cómo funciona toda la economía de un país, mirando todo en general en lugar de solo una parte.
